In [3]:
import os
import tensorflow.compat.v1 as tf
import math
import numpy as np
import itertools
print((tf.__version__))
tf.enable_eager_execution()
import tensorflow
!pip install openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
import cv2
import glob

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset
!pip install pathlib
from pathlib import Path

def transform_laser_labels(objects, transformation):
    labels_transformed = []
    for label in objects:
        w, h = 1, 4;
        transform = [[0 for x in range(w)] for y in range(h)] 
        transform[0] = label.box.center_x
        transform[1] = label.box.center_y
        transform[2] = label.box.center_z
        transform[3] = 1
        np.set_printoptions(suppress=True)
        label_transformed = transformation.dot(transform)
        labels_transformed.append(label_transformed)
    return labels_transformed



waymo_to_labels = {
    0 : 'UNKNOWN',
    1 : 'VEHICLE',
    2 : 'PEDESTRIAN',
    3 : 'SIGN',
    4 : 'CYCLIST'
}

for s in range(0, 8):
    dataset_partition = 'validation_000' + str(s)
    os.makedirs('SampleAnalysis'+ dataset_partition)
    segment = 'validation_000' + str(s)
    path = '/MyTeams/Waymo/WaymoDataset/ValidationSet/' + segment + '/'
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.tfrecord' in file:
                files.append(os.path.join(r, file))


    print(files)
    # file = "/MyTeams/YonohubBlocksRadwan/WaymoDataset/training_0000/segment-10391312872392849784_4099_400_4119_400_with_camera_labels.tfrecord"


    for file in files:
        filename = Path(file).stem
        print("extracting " + filename)
        frames = []
        dataset = tf.data.TFRecordDataset(file, compression_type='')
        for data in dataset:
            frame = open_dataset.Frame()
            frame.ParseFromString(bytearray(data.numpy()))
            frames.append(frame)
        print("frames extracted")
        laser_labels = []
        timestamps = []
        FImages = []
        vehicle_poses = []
        vehicle_speeds = []
        ego_speedx = []
        ego_speedy = []
        locations = []
        weather_states = []
        laser_labels_transformed = []
        prev_pos_x = 0
        prev_pos_y = 0
        prev_timestamp_s = 0

        for i in range(0, len(frames)):
            frame_images = []
            for index, image in enumerate(frames[i].images):
                frame_images.append(tensorflow.image.decode_jpeg(image.image).numpy())

            FImages.append(frame_images[0])

            # laser labels (3d bounding boxes)
            laser_labels.append(frames[i].laser_labels)

            timestamps.append(frames[i].timestamp_micros)

            locations.append(frames[i].context.stats.location)
            weather_states.append(frames[i].context.stats.weather)

            vehicle_poses.append((np.array(frames[i].pose.transform)).reshape((4,4)))
            laser_labels_transformed.append(transform_laser_labels(frames[i].laser_labels, (np.array(frames[i].pose.transform)).reshape((4,4))))
            if i == 0:
                prev_pos_x = vehicle_poses[i][0][3]
                prev_pos_y = vehicle_poses[i][1][3]

            speed_x = (vehicle_poses[i][0][3] - prev_pos_x)/((timestamps[i]/1000000.0 - prev_timestamp_s))
            ego_speedx.append(speed_x)
            speed_y = (vehicle_poses[i][1][3] - prev_pos_y)/((timestamps[i]/1000000.0 - prev_timestamp_s))
            ego_speedy.append(speed_y)
            abs_speed = np.sqrt(speed_x**2 + speed_y**2)
            vehicle_speeds.append(abs_speed)
            prev_pos_x = vehicle_poses[i][0][3]
            prev_pos_y = vehicle_poses[i][1][3]
            prev_timestamp_s = frames[i].timestamp_micros / 1000000.0
        ego_speedx[0] = ego_speedx[1]
        ego_speedy[0] = ego_speedy[1]
        vehicle_speeds[0] = vehicle_speeds[1]
        print("preparing xlsx file")
        headers = ['timestamp_us' ,'frame no', 'tracking ID', 'label', 'objX', 'objY', 'objZ',
                   'absX', 'absY', 'absZ', 'width', 'length', 'height', 'heading', 'objSpeedX', 
                   'objSpeedY', 'objAccX', 'objAccY', 'egoX', 'egoY', 'egoZ', 'egoAbsSpeed', 
                   'egoSpeedX', 'EgoSpeedY', 'Location', 'Weather']
        workbook_name = 'SampleAnalysis'+ dataset_partition + '/' + filename + '.xlsx'
        wb = Workbook()
        page = wb.active
        page.append(headers)

        for i in range(0, len(laser_labels)):
            for x in range(0, len(laser_labels[i])):
                if laser_labels[i][x].type != 3:
                    new_row = [timestamps[i], 
                               i,
                               laser_labels[i][x].id,
                               waymo_to_labels[laser_labels[i][x].type], 
                               laser_labels[i][x].box.center_x, 
                               laser_labels[i][x].box.center_y,
                               laser_labels[i][x].box.center_z,
                               laser_labels_transformed[i][x][0],
                               laser_labels_transformed[i][x][1],
                               laser_labels_transformed[i][x][2],
                               laser_labels[i][x].box.width,
                               laser_labels[i][x].box.length,
                               laser_labels[i][x].box.height,
                               laser_labels[i][x].box.heading,
                               laser_labels[i][x].metadata.speed_x,
                               laser_labels[i][x].metadata.speed_y,
                               laser_labels[i][x].metadata.accel_x,
                               laser_labels[i][x].metadata.accel_y,
                               vehicle_poses[i][0][3],
                               vehicle_poses[i][1][3],
                               vehicle_poses[i][2][3],
                               vehicle_speeds[i],
                               ego_speedx[i],
                               ego_speedy[i],
                               locations[i],
                               weather_states[i]]
                    page.append(new_row)
        print("saving xlsx file")
        wb.save(filename = workbook_name)
        height, width, layers = FImages[0].shape
        size = (width,height)
        out = cv2.VideoWriter('SampleAnalysis'+ dataset_partition + '/' + filename + '.avi',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
        for i in range(len(FImages)):
            out.write(FImages[i])
        print("saving video")
        out.release()





1.15.0
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
You are using pip version 19.0, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
You are using pip version 19.0, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
['/MyTeams/Waymo/WaymoDataset/ValidationSet/validation_0000/segment-10203656353524179475_7625_000_7645_000_with_camera_labels.tfrecord', '/MyTeams/Waymo/WaymoDataset/ValidationSet/validation_0000/segment-1024360143612057520_3580_000_3600_000_with_camera_labels.tfrecord', '/MyTea

frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-12374656037744638388_1412_711_1432_711_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-12496433400137459534_120_000_140_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-12657584952502228282_3940_000_3960_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-12820461091157089924_5202_916_5222_916_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-12831741023324393102_2673_230_2693_230_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-12866817684252793621_480_000_500_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-12940710315541930162_2660_000_2680_000_with_camera_lab

frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-14931160836268555821_5778_870_5798_870_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-14956919859981065721_1759_980_1779_980_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-15021599536622641101_556_150_576_150_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-15028688279822984888_1560_000_1580_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-1505698981571943321_1186_773_1206_773_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-15096340672898807711_3765_000_3785_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-15224741240438106736_960_000_980_000_with_camera_labe

extracting segment-17860546506509760757_6040_000_6060_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-17962792089966876718_2210_933_2230_933_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-18024188333634186656_1566_600_1586_600_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-18045724074935084846_6615_900_6635_900_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
['/MyTeams/Waymo/WaymoDataset/ValidationSet/validation_0004/segment-18252111882875503115_378_471_398_471_with_camera_labels.tfrecord', '/MyTeams/Waymo/WaymoDataset/ValidationSet/validation_0004/segment-18305329035161925340_4466_730_4486_730_with_camera_labels.tfrecord', '/MyTeams/Waymo/WaymoDataset/ValidationSet/validation_0004/segment-18331704533904883545_1560_000_1580_000_with_camera_labels.tfrecord', '/MyTeams/Waymo/W

frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-346889320598157350_798_187_818_187_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-3577352947946244999_3980_000_4000_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-3651243243762122041_3920_000_3940_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-366934253670232570_2229_530_2249_530_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-3731719923709458059_1540_000_1560_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-3915587593663172342_10_000_30_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-4013125682946523088_3540_000_3560_000_with_camera_labels
frames 

preparing xlsx file
saving xlsx file
saving video
extracting segment-6161542573106757148_585_030_605_030_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-6183008573786657189_5414_000_5434_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-6324079979569135086_2372_300_2392_300_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-6491418762940479413_6520_000_6540_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-662188686397364823_3248_800_3268_800_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-6637600600814023975_2235_000_2255_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-6680764940003341232_2260_000_2280_000_with_camera_labels
frames extracted
pr

frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-9114112687541091312_1100_000_1120_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-9164052963393400298_4692_970_4712_970_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-9231652062943496183_1740_000_1760_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-9243656068381062947_1297_428_1317_428_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-9265793588137545201_2981_960_3001_960_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-933621182106051783_4160_000_4180_000_with_camera_labels
frames extracted
preparing xlsx file
saving xlsx file
saving video
extracting segment-9443948810903981522_6538_870_6558_870_with_camera_labels


In [ ]:
import os
import tensorflow.compat.v1 as tf
import math
import numpy as np
import itertools
print((tf.__version__))
tf.enable_eager_execution()
import tensorflow
!pip install openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
import cv2
import glob

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset
!pip install pathlib
from pathlib import Path



waymo_to_labels = {
    0 : 'UNKNOWN',
    1 : 'VEHICLE',
    2 : 'PEDESTRIAN',
    3 : 'SIGN',
    4 : 'CYCLIST'
}


def transform_laser_labels(objects, transformation):
    labels_transformed = []
    for label in objects:
        w, h = 1, 4;
        transform = [[0 for x in range(w)] for y in range(h)] 
        transform[0] = label.box.center_x
        transform[1] = label.box.center_y
        transform[2] = label.box.center_z
        transform[3] = 1
        np.set_printoptions(suppress=True)
        label_transformed = transformation.dot(transform)
        labels_transformed.append(label_transformed)
    return labels_transformed
    

files = ['/MyTeams/Waymo/WaymoDataset/ValidationSet/validation_0000/segment-10868756386479184868_3000_000_3020_000_with_camera_labels.tfrecord',
         '/MyTeams/Waymo/WaymoDataset/ValidationSet/validation_0000/segment-11450298750351730790_1431_750_1451_750_with_camera_labels.tfrecord',
         '/MyTeams/Waymo/WaymoDataset/ValidationSet/validation_0001/segment-13336883034283882790_7100_000_7120_000_with_camera_labels.tfrecord',
         '/MyTeams/Waymo/WaymoDataset/TrainingSet/training_0015/segment-18233614482685846350_7060_000_7080_000_with_camera_labels.tfrecord',
         '/MyTeams/Waymo/WaymoDataset/ValidationSet/validation_0005/segment-4575389405178805994_4900_000_4920_000_with_camera_labels.tfrecord',
         '/MyTeams/Waymo/WaymoDataset/TrainingSet/training_0025/segment-6280779486809627179_760_000_780_000_with_camera_labels.tfrecord',
         '/MyTeams/Waymo/WaymoDataset/TrainingSet/training_0028/segment-7921369793217703814_1060_000_1080_000_with_camera_labels.tfrecord',
         '/MyTeams/Waymo/WaymoDataset/ValidationSet/validation_0006/segment-7932945205197754811_780_000_800_000_with_camera_labels.tfrecord',
         '/MyTeams/Waymo/WaymoDataset/TrainingSet/training_0030/segment-8965112222692085704_4860_000_4880_000_with_camera_labels.tfrecord',
         '/MyTeams/Waymo/WaymoDataset/TrainingSet/training_0031/segment-9907794657177651763_1126_570_1146_570_with_camera_labels.tfrecord']

print(files)

for file in files:
    filename = Path(file).stem
    print("extracting " + filename)
    frames = []
    dataset = tf.data.TFRecordDataset(file, compression_type='')
    for data in dataset:
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        frames.append(frame)
    print("frames extracted")
    laser_labels = []
    timestamps = []
    FImages = []
    vehicle_poses = []
    vehicle_speeds = []
    ego_speedx = []
    ego_speedy = []
    locations = []
    weather_states = []
    laser_labels_transformed = []
    prev_pos_x = 0
    prev_pos_y = 0
    prev_timestamp_s = 0

    for i in range(0, len(frames)):
        frame_images = []
        for index, image in enumerate(frames[i].images):
            frame_images.append(tensorflow.image.decode_jpeg(image.image).numpy())

        FImages.append(frame_images[0])

        # laser labels (3d bounding boxes)
        laser_labels.append(frames[i].laser_labels)
        
        
        timestamps.append(frames[i].timestamp_micros)

        locations.append(frames[i].context.stats.location)
        weather_states.append(frames[i].context.stats.weather)

        vehicle_poses.append((np.array(frames[i].pose.transform)).reshape((4,4)))
        laser_labels_transformed.append(transform_laser_labels(frames[i].laser_labels, (np.array(frames[i].pose.transform)).reshape((4,4))))

        if i == 0:
            prev_pos_x = vehicle_poses[i][0][3]
            prev_pos_y = vehicle_poses[i][1][3]

        speed_x = (vehicle_poses[i][0][3] - prev_pos_x)/((timestamps[i]/1000000.0 - prev_timestamp_s))
        ego_speedx.append(speed_x)
        speed_y = (vehicle_poses[i][1][3] - prev_pos_y)/((timestamps[i]/1000000.0 - prev_timestamp_s))
        ego_speedy.append(speed_y)
        abs_speed = np.sqrt(speed_x**2 + speed_y**2)
        vehicle_speeds.append(abs_speed)
        prev_pos_x = vehicle_poses[i][0][3]
        prev_pos_y = vehicle_poses[i][1][3]
        prev_timestamp_s = frames[i].timestamp_micros / 1000000.0
    ego_speedx[0] = ego_speedx[1]
    ego_speedy[0] = ego_speedy[1]
    vehicle_speeds[0] = vehicle_speeds[1]
    print("preparing xlsx file")
    headers = ['timestamp_us' ,'frame no', 'tracking ID', 'label', 'objX', 'objY', 'objZ', 
               'width', 'length', 'height', 'heading', 'speedX', 'speedY', 'accX', 
                   'accY', 'egoAbsSpeed', 'egoSpeedX', 'egoSpeedY', 'egoX', 'egoY', 'egoZ', 'absX', 'absY', 'absZ']
    workbook_name = 'ShockWaveTransformed/' + filename + '.xlsx'
    wb = Workbook()
    page = wb.active
    page.append(headers)

    for i in range(0, len(laser_labels)):
        for x in range(0, len(laser_labels[i])):
            if laser_labels[i][x].type != 3:
                new_row = [timestamps[i], 
                           i,
                           laser_labels[i][x].id,
                           waymo_to_labels[laser_labels[i][x].type], 
                           laser_labels[i][x].box.center_x, 
                           laser_labels[i][x].box.center_y,
                           laser_labels[i][x].box.center_z,
                           laser_labels[i][x].box.width,
                           laser_labels[i][x].box.length,
                           laser_labels[i][x].box.height,
                           laser_labels[i][x].box.heading,
                           laser_labels[i][x].metadata.speed_x,
                           laser_labels[i][x].metadata.speed_y,
                           laser_labels[i][x].metadata.accel_x,
                           laser_labels[i][x].metadata.accel_y,
                           vehicle_speeds[i],
                           ego_speedx[i],
                           ego_speedy[i],
                           vehicle_poses[i][0][3],
                           vehicle_poses[i][1][3],
                           vehicle_poses[i][2][3],
                           laser_labels_transformed[i][x][0],
                           laser_labels_transformed[i][x][1],
                           laser_labels_transformed[i][x][2]]
                page.append(new_row)
    print("saving xlsx file")
    wb.save(filename = workbook_name)
    height, width, layers = FImages[0].shape
    size = (width,height)
    out = cv2.VideoWriter('ShockWaveTransformed/' + filename + '.avi',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
    for i in range(len(FImages)):
        out.write(FImages[i])
    print("saving video")
    out.release()





In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
print(vehicle_poses[0])
print(laser_labels[0][0].box)
for label in laser_labels[0]:
    w, h = 1, 4;
    transform = [[0 for x in range(w)] for y in range(h)] 
    transform[0] = label.box.center_x
    transform[1] = label.box.center_y
    transform[2] = label.box.center_z
    transform[3] = 1
    np.set_printoptions(suppress=True)
    label_transformed = vehicle_poses[0].dot(transform)
    
    print(label_transformed.shape)
#     plt.plot(0, 0, marker=">")
    plt.plot(vehicle_poses[0][0][3], vehicle_poses[0][1][3], marker=">")
    plt.plot(label_transformed[0], label_transformed[1], marker="+")
#     plt.xlim([-26000, -25500])
#     plt.ylim([40230, 40250])
    plt.show()
    break
    

In [ ]:
print(frames[0].pose.transform)